<a href="https://colab.research.google.com/github/sheetal1996/Eco-lution/blob/main/575DataCleanign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import random
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [1]:
#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/
from google.colab import files
 
 
uploaded = files.upload()

Saving owid-co2-data.csv to owid-co2-data.csv


In [3]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['owid-co2-data.csv']))
print(df. columns) 

Index(['iso_code', 'country', 'year', 'co2', 'co2_per_capita', 'trade_co2',
       'cement_co2', 'cement_co2_per_capita', 'coal_co2',
       'coal_co2_per_capita', 'flaring_co2', 'flaring_co2_per_capita',
       'gas_co2', 'gas_co2_per_capita', 'oil_co2', 'oil_co2_per_capita',
       'other_industry_co2', 'other_co2_per_capita', 'co2_growth_prct',
       'co2_growth_abs', 'co2_per_gdp', 'co2_per_unit_energy',
       'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_co2', 'cumulative_cement_co2',
       'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'trade_co2_share',
       'share_global_co2', 'share_global_cement_co2', 'share_global_coal_co2',
       'share_global_flaring_co2', 'share_global_gas_co2',
       'share_global_oil_co2', 'share_global_other_co2',
       'share_global_cumulative_co2', 'share_global_cumulative_cement_co2',
       'share_global_cumulative_c

In [ ]:
df.shape

In [5]:
#Let us first find out the columns which have atleast 50% of non-null row
df.columns[df.notnull().mean() >= 0.5]



Index(['iso_code', 'country', 'year', 'co2', 'co2_per_capita', 'coal_co2',
       'coal_co2_per_capita', 'oil_co2', 'oil_co2_per_capita',
       'co2_growth_prct', 'co2_growth_abs', 'co2_per_gdp', 'cumulative_co2',
       'cumulative_coal_co2', 'cumulative_oil_co2', 'share_global_co2',
       'share_global_coal_co2', 'share_global_oil_co2',
       'share_global_cumulative_co2', 'share_global_cumulative_coal_co2',
       'share_global_cumulative_oil_co2', 'population', 'gdp'],
      dtype='object')

In [22]:
#Since we need to remove the other columns with less than 50% of non-null rows, we use the 'drop' function
#we assign the resulting dataframe to a new variable name 'df_dropcol'
df_dropcol = df.drop(df.columns[df.notnull().mean() < 0.5],axis=1)

#Now, if we see again the information about this new df_dropcol dataframe (by using the info function), 
#we can see that we still have null values. So, in order to remove these null values, we can use dropna() function on df_dropcol.

df_dropcol.dropna(inplace=True)
df_dropcol.shape

(8880, 23)

> Step 1: We have to find a way to fill the null values.
If we are doing this, we are losing 2020 data as well. Even if we are directly removing the null values using dropna(), we are losing even higher data.


In [18]:
df_dropcol

,iso_code,country,year,co2,co2_per_capita,coal_co2,coal_co2_per_capita,oil_co2,oil_co2_per_capita,co2_growth_prct,...,cumulative_coal_co2,cumulative_oil_co2,share_global_co2,share_global_coal_co2,share_global_oil_co2,share_global_cumulative_co2,share_global_cumulative_coal_co2,share_global_cumulative_oil_co2,population,gdp
1,AFG,Afghanistan,1950,0.084,0.011,0.021,0.003,0.063,0.008,475.00,...,0.036,0.063,0.00,0.00,0.00,0.00,0.00,0.00,7752117.0,9.421400e+09
2,AFG,Afghanistan,1951,0.092,0.012,0.026,0.003,0.066,0.008,8.70,...,0.061,0.129,0.00,0.00,0.00,0.00,0.00,0.00,7840151.0,9.692280e+09
3,AFG,Afghanistan,1952,0.092,0.012,0.032,0.004,0.060,0.008,0.00,...,0.093,0.189,0.00,0.00,0.00,0.00,0.00,0.00,7935996.0,1.001733e+10
4,AFG,Afghanistan,1953,0.106,0.013,0.038,0.005,0.068,0.008,16.00,...,0.131,0.257,0.00,0.00,0.00,0.00,0.00,0.00,8039684.0,1.063052e+10
5,AFG,Afghanistan,1954,0.106,0.013,0.043,0.005,0.064,0.008,0.00,...,0.174,0.321,0.00,0.00,0.00,0.00,0.00,0.00,8151316.0,1.086636e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25982,ZWE,Zimbabwe,2014,11.906,0.876,7.691,0.566,3.719,0.274,0.59,...,570.112,126.428,0.03,0.05,0.03,0.05,0.08,0.02,13586710.0,2.122250e+10
25983,ZWE,Zimbabwe,2015,12.226,0.885,8.033,0.582,3.608,0.261,2.69,...,578.145,130.036,0.03,0.05,0.03,0.05,0.08,0.02,13814642.0,2.102745e+10
25984,ZWE,Zimbabwe,2016,10.738,0.765,6.959,0.496,3.139,0.224,-12.17,...,585.104,133.175,0.03,0.05,0.03,0.05,0.08,0.02,14030338.0,2.096179e+10
25985,ZWE,Zimbabwe,2017,9.582,0.673,5.665,0.398,3.239,0.228,-10.77,...,590.768,136.414,0.03,0.04,0.03,0.05,0.08,0.02,14236599.0,2.194784e+10


In [17]:
df_gdp_2018 = df_dropcol.loc[df_dropcol['year']==2020] # finds all the countries with data based on 201

#df_gdp_top10 = df_gdp_2018.sort_values(by=['gdp'], ascending = False).head(10) #arranges the countries based on their GDP in 2020

#print(df_gdp_2018['country']) # prints the top 10 countries with highest GDP8
print(df_gdp_2018.to_markdown())

| iso_code   | country   | year   | co2   | co2_per_capita   | coal_co2   | coal_co2_per_capita   | oil_co2   | oil_co2_per_capita   | co2_growth_prct   | co2_growth_abs   | co2_per_gdp   | cumulative_co2   | cumulative_coal_co2   | cumulative_oil_co2   | share_global_co2   | share_global_coal_co2   | share_global_oil_co2   | share_global_cumulative_co2   | share_global_cumulative_coal_co2   | share_global_cumulative_oil_co2   | population   | gdp   |
|------------|-----------|--------|-------|------------------|------------|-----------------------|-----------|----------------------|-------------------|------------------|---------------|------------------|-----------------------|----------------------|--------------------|-------------------------|------------------------|-------------------------------|------------------------------------|-----------------------------------|--------------|-------|


In [27]:
df_nona = df.dropna()

#Investigating the shape of 'df_nona'

df_nona.shape

#df_nona['year'].max() # This line of code will return the most recent year in the data set

df_dropcol['year'].max()

2018